In [4]:
!git clone https://github.com/katulevskiy/NLP-OpenBookQA.git

fatal: destination path 'NLP-OpenBookQA' already exists and is not an empty directory.


In [18]:
'''
Imports
'''

from sentence_transformers import SentenceTransformer, util
import torch

In [82]:
'''
Load model + list of facts, get fact embeddings. for now just using all-mpbnet-base-v2
'''

model_name = "all-mpnet-base-v2"
model = SentenceTransformer(model_name)

facts = []
facts_file = open("/content/NLP-OpenBookQA/OpenBookQA-V1-Sep2018/Data/Main/openbook.txt","r")
for fact in facts_file.readlines():
  facts.append(fact[1:len(fact)-2])

fact_embeddings = model.encode(facts, convert_to_tensor=True)

In [19]:
torch.save(fact_embeddings,"fact_embeddings.pt")

In [97]:
def get_top_k_facts(query,embedding_model,facts,fact_embeddings,k=3, per_answer=False):
  if not per_answer:
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, fact_embeddings)[0]
    top_k_indices = cos_scores.topk(k=k).indices
    results = [facts[idx] for idx in top_k_indices]
    return results

  queries = []
  question = query.split('(A)')[0][:-1]

  answer_a = query.split('(A)')[1].split('(B)')[0][:-1]
  queries.append(question + answer_a)
  answer_b = query.split('(B)')[1].split('(C)')[0][:-1]
  queries.append(question + answer_b)
  answer_c = query.split('(C)')[1].split('(D)')[0][:-1]
  queries.append(question + answer_c)
  answer_d = query.split('(D)')[1]
  queries.append(question + answer_d)

  results = []
  for query in queries:
    query_embedding = embedding_model.encode(query, convert_to_tensor=True)
    cos_scores = util.cos_sim(query_embedding, fact_embeddings)[0]
    top_k_indices = cos_scores.topk(k=k).indices
    for idx in top_k_indices:
      results.append(facts[idx])
      fact_embeddings = torch.cat((fact_embeddings[:idx], fact_embeddings[idx+1:]))

  return results

In [72]:
fact_embeddings = torch.load("fact_embeddings.pt",weights_only=True)

query = "The sun is responsible for	(A) puppies learning new tricks (B) children growing up and getting old (C) flowers wilting in a vase (D) plants sprouting, blooming and wilting"

print("Getting top 3 for each answer choice:\n")
for fact in get_top_k_facts(query,model,facts,fact_embeddings,per_answer=True):
  print(fact)

print("\n\n")
print("Getting top 12 for query with all answer choices:\n")
for fact in get_top_k_facts(query,model,facts,fact_embeddings,k=12):
  print(fact)


Getting top 3 for each answer choice:

the sun is the source of energy for life on Earth
the sun is the source of solar energy called sunlight
the sun is a source of light called sunlight
the sun is located directly overhead at noon
the sun is the source of energy for life on Earth
the sun is a source of heat called sunlight
a plant requires sunlight to grow
a plant requires sunlight for photosynthesis
if a tree falls then sunlight becomes available to the surrounding plants
a leaf absorbs sunlight to perform photosynthesis
photosynthesis means green plants convert from carbon dioxide, water, and solar energy into oxygen for themselves
a plant light is used for help plants by mimicking sunlight



Getting top 12 for query with all answer choices:

the sun is the source of energy for life on Earth
the sun is the source of energy for physical cycles on Earth
the sun is the source of solar energy called sunlight
the sun is a source of light called sunlight
the sun is a source of heat call

In [83]:
query = "When standing miles away from Mount Rushmore (A) the mountains seem very close (B) the mountains are boring (C) the mountains look the same as from up close (D) the mountains seem smaller than in photographs"

print("Getting top 3 for each answer choice:\n")
for fact in get_top_k_facts(query,model,facts,fact_embeddings,per_answer=True):
  print(fact)

print("\n\n")
print("Getting top 12 for query with all answer choices:\n")
for fact in get_top_k_facts(query,model,facts,fact_embeddings,k=12):
  print(fact)

Getting top 3 for each answer choice:

as distance from an object decreases , that object will appear larger
the surface of the Earth contains mountains
if an object is close then that object will appear large
the sun transfers heat energy from itself to the planets through sunlight
motion is a source of kinetic energy in an object
landfills have a negative impact on the communities
telephones convert sound energy into electrical energy
if an object is blue then that object reflects only blue light
as distance to a city decreases , the amount of light pollution will increase
as the size of a leaf increases , the amount of sunlight absorbed by that leaf will increase
as the size of a flower increases , the number of pollinators it will attract increases
loss of resources has a negative impact on the organisms in an area



Getting top 12 for query with all answer choices:

the surface of the Earth contains mountains
the surface of the Moon contains mountains
as distance from an object d

In [81]:
query = "When food is reduced in the stomach (A) the mind needs time to digest (B) take a second to digest what I said (C) nutrients are being deconstructed (D) reader's digest is a body of works"

print("Getting top 3 for each answer choice:\n")
for fact in get_top_k_facts(query,model,facts,fact_embeddings,per_answer=True):
  print(fact)

print("\n\n")
print("Getting top 12 for query with all answer choices:\n")
for fact in get_top_k_facts(query,model,facts,fact_embeddings,k=12):
  print(fact)

Getting top 3 for each answer choice:

if food is not immediately used by the body for energy then that food will be stored for future use
digestion is when stomach acid breaks down food
the digestive system digests food for the body
the arctic environment is white in color from being covered in snow
the decrease of something required by an organism has a negative impact on that organism 's survival
the crust is a layer of the Earth
the continental shelf is located near the shore
digestion is when an organism takes in nutrients from food into itself by eating
cellular respiration is when cells break down food to produce energy
eagles eat rabbits
bacteria can help digest food in humans
flowing liquid can push objects



Getting top 12 for query with all answer choices:

digestion is when stomach acid breaks down food
the digestive system digests food for the body
the breaking down of food into simple substances occurs in the digestive system
digestion is when an organism takes in nutrie

In [86]:
from sentence_transformers import CrossEncoder

cross_model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [107]:
def top_k_facts_cross(query,embedding_model,cross_model,facts,fact_embeddings,k=5,n=50):
  top_n = get_top_k_facts(query,embedding_model,facts,fact_embeddings,k=n)
  ranks = cross_model.rank(query,top_n)
  results = [top_n[rank['corpus_id']] for rank in ranks[0:k]]
  return results


In [108]:
query = "When standing miles away from Mount Rushmore (A) the mountains seem very close (B) the mountains are boring (C) the mountains look the same as from up close (D) the mountains seem smaller than in photographs"
for fact in top_k_facts_cross(query,model,cross_model,facts,fact_embeddings,k=12):
  print(fact)

as a source of light becomes closer , that source will appear brighter
mountains are formed by volcanoes
sometimes weathering rounds rocks
as distance from a source of light increases , that source of light will appear dimmer
if an object is close then that object will appear large
mountains are formed by plate tectonics
landslides often occur on mountains
as distance to an object increases , that object will appear smaller
as altitude increases , air pressure will decrease
a tape measure is used to measure distance
as depth increases , pressure will increase
as distance from an object decreases , that object will appear larger
